![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Implementing hybrid search with Redis

Hybrid search is all about combining lexical search with semantic vector search to improve result relevancy. This notebook will cover 3 different hybrid search strategies with Redis:

1. Linear combination of scores from lexical search (BM25) and vector search (Cosine Distance) with the HybridQuery class
2. Client-Side Reciprocal Rank Fusion (RRF)
3. Client-Side Reranking with a cross encoder model

>Note: Additional work is planed within Redis Query Engine core to add more flexible hybrid search capabilities in the future.

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/02_hybrid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Install Packages

In [ ]:
%pip install sentence-transformers pandas nltk "git+https://github.com/redis/redis-vl-python.git@0.6.0"

### Data/Index Preparation
 
In this section:

1. We prepare the data necessary for our hybrid search implementations by loading a collection of movies. Each movie object contains the following attributes:
    - `title`
    - `rating`
    - `description`
    - `genre`
 
2. We generate vector embeddings from the movie descriptions. This allows users to perform searches that not only rely on exact matches but also on semantic relevance, helping them find movies that align closely with their interests.

3. After preparing the data, we populate a search index with these movie records, enabling efficient querying based on both lexical and vector-based search techniques.

Running remotely or in collab? Run this cell to download the necessary dataset.

In [ ]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/vector-search/resources .
!rm -rf temp_repo

### Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings and full text fields. **We need to have a Redis
instance available.**

#### Local Redis
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### Alternative Redis Access (Cloud, Docker, other)
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [1]:
import os
import warnings

warnings.filterwarnings('ignore')

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create redis client, load data, generate embeddings

In [2]:
from redis import Redis

client = Redis.from_url(REDIS_URL)

In [3]:
import json

with open("resources/movies.json", 'r') as file:
    movies = json.load(file)

In [4]:
from redisvl.utils.vectorize import HFTextVectorizer
from redisvl.extensions.cache.embeddings import EmbeddingsCache


# load model for embedding our movie descriptions
model = HFTextVectorizer(
    model='sentence-transformers/all-MiniLM-L6-v2',
    cache=EmbeddingsCache(
        name="embedcache",
        ttl=600,
        redis_client=client,
    )
)

# embed movie descriptions
movie_data = [
    {
        **movie,
        "description_vector": model.embed(movie["description"], as_buffer=True, dtype="float32")
    } for movie in movies
]

In [5]:
movie_data[:1]

[{'title': 'Explosive Pursuit',
  'genre': 'action',
  'rating': 7,
  'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.',
  'description_vector': b'\x91f|=\xb6`\n;g\x92\xb7;3\xcb~\xbd\x16e\xce\xbb\xd7\x16J=P\xa7?=\xc8v\x95<i\xfa\x06\xbe\x12Y\xcf=4\x07p=D\xdb\r\xbd\x8d\xf2H\xbdfe\xc6<G\xdfa=t8\x16\xbc\xd4\xd3\x13<A\xaa\x1c=\x06\xef\x89<\xb6\xb0-<\x99\xb2\x9f\xbcZ\x0b\xc3\xbd\xa5NR=Zl\xf7\xbcN>\x17\xbe\x02\x1a\x05\xb9@u\xbf<\xd6\xe2b\xba\xd0\xa6\xa8\xbdo\xdc\xec\xbcQc%=N\xe7r\xbb\x1dOG==(\x85=y@\xa2\xbc7Z\xd0\xbdB%K\xbd\xba\xed\x94\xbcU\xddH=\xbe&F<\xbc*\xec<\x8c\xd8\x8d\xbd\xf3Z\x98<\x15\xa3\xa3=3g3\xbd$\xcd\xbd\xbd\xf7$\xf7;\xf6\xf4z=\x02\xb5\x8c=\x8d\x0e\xc6\xbdhI\x90\xbdq\x16\xbd;u\xe7\x0c\xbd&3\xc9\xbc\x82\xf8\xbb\xbc\xa7&u\xbb-\x8f\xca<\xf2\x7fJ=\x14\xaf*=\x87OU\xbd\xde\xf0\x95\xbc \x02\x19=\x1b\xf4K<\xd0\xc2\t=F\x83\xac=\x9e\xd7\xb8\xbd\xf3\xb5\x9c\xbdB\x85\x18=\xa4d&=\'3\xf8<\xd3\xf7\x88<Tv\xf2\xbb1=[\xbda\xac\xee\xbb

### Define Redis index schema

Below, we build a schema that represents our movie objects.

In [6]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex


schema = IndexSchema.from_dict({
  "index": {
    "name": "movies",
    "prefix": "movie",
    "storage": "hash"
  },
  "fields": [
    { "name": "title", "type": "text" },
    { "name": "description", "type": "text" },
    { "name": "genre", "type": "tag", "attrs": {"sortable": True}},
    { "name": "rating", "type": "numeric", "attrs": {"sortable": True}},
    {
        "name": "description_vector",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "hnsw",
            "datatype": "float32"
        }
    }
  ]
})


index = SearchIndex(schema, client, validate_on_load=True)
index.create(overwrite=True, drop=True)

### Populate index

Load movie objects into Redis

In [7]:
index.load(movie_data)

['movie:01JR6XHSRXW6DF4TSF7N2KJD21',
 'movie:01JR6XHSS0HK3E95HDVE1D5Y4W',
 'movie:01JR6XHSS0EE89YNT702SB1ZZN',
 'movie:01JR6XHSS09BNDBRZHA6F5RT0H',
 'movie:01JR6XHSS0N8W3M4YDR45HHCPY',
 'movie:01JR6XHSS1XDQFJXHHJD9Z0AW3',
 'movie:01JR6XHSS17RNERVEQ9YAHA81D',
 'movie:01JR6XHSS182SYE6E630Y6D6B1',
 'movie:01JR6XHSS1899GWQDBWHP0Q57A',
 'movie:01JR6XHSS1WRCF39MX0XQ60PEJ',
 'movie:01JR6XHSS1K9RYSRJTTNRQQP5C',
 'movie:01JR6XHSS148EGMV6JTECWRC5J',
 'movie:01JR6XHSS1199WGN103VB7GGHE',
 'movie:01JR6XHSS12RTJR8B953SDWC1E',
 'movie:01JR6XHSS1WR63V3MZNVZ5X82R',
 'movie:01JR6XHSS2Z1F2C0ZG60FCJ36F',
 'movie:01JR6XHSS250RZBG2M3C3VKWE2',
 'movie:01JR6XHSS2ECNNK19FRAZ00RE6',
 'movie:01JR6XHSS2W46EMH9EVC42TBVV',
 'movie:01JR6XHSS2K8KF6DK6MZN65MT8']

# Hybrid Search Approaches

Now that our search index is populated and ready, we will build out a few different hybrid search techniques in Redis.

To start, we will use our `HybridQuery` class that accepts a text string and vector to automatically combine text similarity and vector similarity scores.

## 1. Linear Combination using HybridQuery

The goal of this technique is to calculate a weighted sum of the text similarity score for our provided text search and the cosine distance between vectors calculated via a KNN vector query. Under the hood this is possible in Redis using the [aggregations API](https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/aggregations/), as of `Redis 7.4.x` (search version `2.10.5`), within a single database call.

As of RedisVl 0.5.0 all of this is nicely encapsulated in your `HybridQuery` class, which behaves much like our other query classes.

In [8]:
# Sample user query (can be changed for comparisons)
user_query = "action adventure movie with great fighting scenes against a dangerous criminal, crime busting, superheroes, and magic"

First, we will import our `HybridQuery` and understand its parameters.
At a minimum, the `HybridQuery` needs 4 arguments:
```python
query = HybridQuery(
    text = "your query string here",
    text_field_name = "<name of the text field in the index to do text search in>",
    vector = <bytes or numeric array, ex: [0.1, 0.2, 0.3]>,
    vector_field_name = "<name of the vector field in the index to compute vector similarity>",
)
```

In [12]:
from redisvl.query import HybridQuery

vector = model.embed(user_query, as_buffer=True, dtype="float32")

query = HybridQuery(
    text=user_query,
    text_field_name="description",
    vector=vector,
    vector_field_name="description_vector",
    return_fields=["title"],
)

results = index.query(query)

results[:3]

[{'vector_distance': '0.645975470543',
  'title': 'The Incredibles',
  'vector_similarity': '0.677012264729',
  'text_score': '10.5386477145',
  'hybrid_score': '3.63550289966'},
 {'vector_distance': '0.797545194626',
  'title': 'Skyfall',
  'vector_similarity': '0.601227402687',
  'text_score': '4.73920856087',
  'hybrid_score': '1.84262175014'},
 {'vector_distance': '0.608649492264',
  'title': 'Explosive Pursuit',
  'vector_similarity': '0.695675253868',
  'text_score': '3.93239518818',
  'hybrid_score': '1.66669123416'}]

That's it! That is all it takes to perform a hybrid text matching and vector query with RedisVL.
Of course there are many more configurations and things we can do with the `HybridQuery` class. Let's investigate.

First, let's look at just the text query part that is being run:

In [13]:
query._build_query_string()

'(~@description:(action | adventure | movie | great | fighting | scenes | dangerous | criminal | crime | busting | superheroes | magic))=>[KNN 10 @description_vector $vector AS vector_distance]'

### Choosing your stopwords for better queries
You can see that the user query string has been tokenized and certain stopwords like 'and', 'for', 'with', 'but', have been removed, otherwise you would get matches on irrelevant words.
RedisVL uses [NLTK](https://www.nltk.org/index.html) english stopwords as the the default. You can change which default language stopwords to use with the `stopwords` argument.
You specify a language, like 'german', 'arabic', 'greek' and many others, provide your own list of stopwords, or set it to `None` to not remove any.

In [14]:
# translate our user query to French and use nltk french stopwords
french_query_text = "Film d'action et d'aventure avec de superbes scènes de combat, des enquêtes criminelles, des super-héros et de la magie"

french_film_query = HybridQuery(
    text=french_query_text,
    text_field_name="description",
    vector=model.embed(french_query_text, as_buffer=True, dtype="float32"),
    vector_field_name="description_vector",
    stopwords="french",
)

print(french_film_query._build_query_string())

# specify your own stopwords
custom_stopwords = set([
    "a", "is", "the", "an", "and", "are", "as", "at", "be", "but", "by", "for",
    "if", "in", "into", "it", "no", "not", "of", "on", "or", "such", "that", "their",
    "then", "there", "these", "they", "this", "to", "was", "will", "with"
])

stopwords_query = HybridQuery(
    text=user_query,
    text_field_name="description",
    vector=vector,
    vector_field_name="description_vector",
    stopwords=custom_stopwords,
)

print(stopwords_query._build_query_string())

# don't use any stopwords
no_stopwords_query = HybridQuery(
    text=user_query,
    text_field_name="description",
    vector=vector,
    vector_field_name="description_vector",
    stopwords=None,
)

print(no_stopwords_query._build_query_string())

(~@description:(film | d\'action | d\'aventure | superbes | scènes | combat | enquêtes | criminelles | super\-héros | magie))=>[KNN 10 @description_vector $vector AS vector_distance]
(~@description:(action | adventure | movie | great | fighting | scenes | against | dangerous | criminal | crime | busting | superheroes | magic))=>[KNN 10 @description_vector $vector AS vector_distance]
(~@description:(action | adventure | movie | with | great | fighting | scenes | against | a | dangerous | criminal | crime | busting | superheroes | and | magic))=>[KNN 10 @description_vector $vector AS vector_distance]


### Choosing your text scoring function and weights
There are different ways to calculate the similarity between sets of text. Redis supports several, such as `BM25`, `TFIDF`, `DISMAX`, and others. The default is `BM25STD` and is easy to configure with the `text_scorer` parameter. Just like changing you embedding model can change your vector similarity scores, changing your text similarity measure can change your text scores.

Because hybrid queries are performing a weighted average of text similarity and vector similarity you also control the relative balance of these scores with the `alpha` parameter.

The documents are ranked based on the hybrid score which is computed as:

```python
hybrid_score = {1-alpha} * text_score + {alpha} * vector_similarity
```

Try changing the `text_scorer` and `alpha` parameters in the query below to see how results may change.


In [15]:
tfidf_query = HybridQuery(
    text=user_query,
    text_field_name="description",
    vector=vector,
    vector_field_name="description_vector",
    text_scorer="TFIDF", # can be one of [TFIDF, TFIDF.DOCNORM, BM25, DISMAX, DOCSCORE, BM25STD]
    stopwords=None,
    alpha=0.25, # weight the vector score lower
    return_fields=["title", "description"],
)

results = index.query(tfidf_query)

results[:3]

[{'vector_distance': '0.645975470543',
  'title': 'The Incredibles',
  'description': "A family of undercover superheroes, while trying to live the quiet suburban life, are forced into action to save the world. Bob Parr (Mr. Incredible) and his wife Helen (Elastigirl) were among the world's greatest crime fighters, but now they must assume civilian identities and retreat to the suburbs to live a 'normal' life with their three children. However, the family's desire to help the world pulls them back into action when they face a new and dangerous enemy.",
  'vector_similarity': '0.677012264729',
  'text_score': '6',
  'hybrid_score': '4.66925306618'},
 {'vector_distance': '0.653376281261',
  'title': 'The Dark Knight',
  'description': 'Batman faces off against the Joker, a criminal mastermind who threatens to plunge Gotham into chaos.',
  'vector_similarity': '0.673311859369',
  'text_score': '4',
  'hybrid_score': '3.16832796484'},
 {'vector_distance': '0.608649492264',
  'title': 'Expl

## 2. Client-side fusion with RRF

Instead of relying on document scores like cosine similarity and BM25/TFIDF, we can fetch items and focus on their rank. This rank can be utilized to create a new ranking metric known as [Reciprocal Rank Fusion (RRF)](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf). RRF is powerful because it can handle ranked lists of different length, scores of different scales, and other complexities.

Although Redis does not currently support RRF natively, we can easily implement it on the client side.

In [14]:
def fuse_rankings_rrf(*ranked_lists, weights=None, k=60):
    """
    Perform Weighted Reciprocal Rank Fusion on N number of ordered lists.
    """
    item_scores = {}
    
    if weights is None:
        weights = [1.0] * len(ranked_lists)
    else:
        assert len(weights) == len(ranked_lists), "Number of weights must match number of ranked lists"
        assert all(0 <= w <= 1 for w in weights), "Weights must be between 0 and 1"
    
    for ranked_list, weight in zip(ranked_lists, weights):
        for rank, item in enumerate(ranked_list, start=1):
            if item not in item_scores:
                item_scores[item] = 0
            item_scores[item] += weight * (1 / (rank + k))
    
    # Sort items by their weighted RRF scores in descending order
    return sorted(item_scores.items(), key=lambda x: x[1], reverse=True)

In [15]:
# Below is a simple example of RRF over a few lists of numbers
fuse_rankings_rrf([1, 2, 3], [2, 4, 6, 7, 8], [5, 6, 1, 2])

[(2, 0.04814747488101534),
 (1, 0.032266458495966696),
 (6, 0.03200204813108039),
 (5, 0.01639344262295082),
 (4, 0.016129032258064516),
 (3, 0.015873015873015872),
 (7, 0.015625),
 (8, 0.015384615384615385)]

We'll want some helper functions to  construct our individual text and vector queries

In [21]:
# Function to create a vector query using RedisVL helpers for ease of use
from redisvl.query import VectorQuery, TextQuery


def make_vector_query(user_query: str, num_results: int, filters = None) -> VectorQuery:
    """Generate a Redis vector query given user query string."""
    vector = model.embed(user_query, as_buffer=True, dtype="float32")
    query = VectorQuery(
        vector=vector,
        vector_field_name="description_vector",
        num_results=num_results,
        return_fields=["title", "description"]
    )
    if filters:
        query.set_filter(filters)
    return query


def make_ft_query(text_field: str, user_query: str, num_results: int) -> TextQuery:
    """Generate a Redis full-text query given a user query string."""
    return TextQuery(
        text=user_query,
        text_field_name=text_field,
        text_scorer="BM25",
        num_results=num_results,
        return_fields=["title", "description"],
    )

In [22]:
from typing import List, Dict, Any


def weighted_rrf(
    user_query: str,
    alpha: float = 0.5,
    num_results: int = 4,
    k: int = 60,
) -> List[Dict[str, Any]]:
    """Implemented client-side RRF after querying from Redis."""
    # Create the vector query
    vector_query = make_vector_query(user_query, num_results=len(movie_data))

    # Create the full-text query
    full_text_query = make_ft_query("description", user_query, num_results=len(movie_data))

    # Run queries individually
    vector_query_results = index.query(vector_query)
    full_text_query_results = index.query(full_text_query)

    # Extract titles from results
    vector_titles = [movie["title"] for movie in vector_query_results]
    full_text_titles = [movie["title"] for movie in full_text_query_results]

    # Perform weighted RRF
    return fuse_rankings_rrf(vector_titles, full_text_titles, weights=[alpha, 1-alpha], k=k)[:num_results]

In [23]:
# Test it out!
weighted_rrf(user_query, num_results=6)

[('Explosive Pursuit', 0.01639344262295082),
 ('The Dark Knight', 0.015873015873015872),
 ('Despicable Me', 0.015625),
 ('The Incredibles', 0.015417457305502846),
 ('Skyfall', 0.0152073732718894),
 ('Finding Nemo', 0.014242424242424244)]

But say we want to give more weight to the vector search rankings in this case to boost semantic similarities contribution to the final rank:

In [24]:
weighted_rrf(user_query, alpha=0.7, num_results=6)

[('Explosive Pursuit', 0.01639344262295082),
 ('The Dark Knight', 0.015873015873015872),
 ('The Incredibles', 0.015702087286527514),
 ('Despicable Me', 0.015625),
 ('Skyfall', 0.014838709677419354),
 ('Finding Nemo', 0.01387878787878788)]

## 3. Client-side reranking

An alternative approach to RRF is to simply use an external reranker to order the final recommendations. RedisVL has built-in integrations to a few popular reranking modules.

In [25]:
from redisvl.utils.rerank import HFCrossEncoderReranker

# Load the ms marco MiniLM cross encoder model from huggingface
reranker = HFCrossEncoderReranker("cross-encoder/ms-marco-MiniLM-L-6-v2")


def rerank(
    user_query: str,
    num_results: int = 4,
) -> List[Dict[str, Any]]:
    """Rerank the candidates based on the user query with an external model/module."""
    # Create the vector query
    vector_query = make_vector_query(user_query, num_results=num_results)

    # Create the full-text query
    full_text_query = make_ft_query("description", user_query, num_results=num_results)

    # Run queries individually
    vector_query_results = index.query(vector_query)
    full_text_query_results = index.query(full_text_query)

    # Assemble list of potential movie candidates with their IDs
    movie_map = {}
    for movie in vector_query_results + full_text_query_results:
        candidate = f"Title: {movie['title']}. Description: {movie['description']}"
        if candidate not in movie_map:
            movie_map[candidate] = movie

    # Rerank candidates
    reranked_movies, scores = reranker.rank(
        query=user_query,
        docs=list(movie_map.keys()),
        limit=num_results,
        return_score=True
    )

    # Fetch full movie objects for the reranked results
    return [
        (movie_map[movie['content']]["title"], score)
        for movie, score in zip(reranked_movies, scores)
    ]


In [26]:
# Test it out!
rerank(user_query, num_results=6)

[('The Incredibles', -4.1636810302734375),
 ('Explosive Pursuit', 0.8551048636436462),
 ('The Dark Knight', -4.403156280517578),
 ('Skyfall', -7.830077171325684),
 ('Mad Max: Fury Road', -7.7119951248168945),
 ('Despicable Me', -8.742403030395508)]

This technique is certainly much slower than simple RRF as it's running an additional cross-encoder model to rerank the results. This can be fairly computationally expensive, but tunable with enough clarity on the use case and focus (how many items to retrieve? how many items to rerank? model accleration via GPU?)

## Comparing Approaches

While each approach has strengths and weaknesses, it's important to understand that each might work better in some use cases than others. Below we will run through a sample of user queries and generate matches for each using different hybrid search techniques.

In [27]:
movie_user_queries = [
    "I'm in the mood for a high-rated action movie with a complex plot",
    "What's a funny animated film about unlikely friendships?",
    "Any movies featuring superheroes or extraordinary abilities", 
    "I want to watch a thrilling movie with spies or secret agents",
    "Are there any comedies set in unusual locations or environments?",
    "Find me an action-packed movie with car chases or explosions",
    "What's a good family-friendly movie with talking animals?",
    "I'm looking for a film that combines action and mind-bending concepts",
    "Suggest a movie with a strong female lead character",
    "What are some movies that involve heists or elaborate plans?",
    "I need a feel-good movie about personal growth or transformation",
    "Are there any films that blend comedy with action elements?", 
    "Show me movies set in dystopian or post-apocalyptic worlds",
    "I'm interested in a movie with themes of revenge or justice",
    "What are some visually stunning movies with impressive special effects?"
]

In [28]:
def hybrid_query(text, alpha, num_results) -> List[Dict[str, Any]]:

    query = HybridQuery(
        text,
        text_field_name="description",
        vector=model.embed(text, as_buffer=True, dtype="float32"),
        vector_field_name="description_vector",
        text_scorer="BM25",
        stopwords="english",
        alpha=alpha,
        return_fields=["title", "hybrid_score"],
    )

    results =  index.query(query)

    return [
        (
            movie["title"],
            movie["hybrid_score"]
        )
        for movie in results
    ]

In [29]:
import pandas as pd


rankings = pd.DataFrame()
rankings["queries"] = movie_user_queries

# First, add new columns to the DataFrame
rankings["hf-cross-encoder"] = ""
rankings["rrf"] = ""
rankings["linear-combo-bm25-cosine"] = ""

In [30]:
# Now iterate through the queries and add results
for i, user_query in enumerate(movie_user_queries):
    rankings.at[i, "hf-cross-encoder"] = rerank(user_query, num_results=4)
    rankings.at[i, "rrf"] = weighted_rrf(user_query, alpha=0.7, num_results=4)
    rankings.at[i, "linear-combo-bm25-cosine"] = hybrid_query(user_query, alpha=0.7, num_results=4)

In [31]:
rankings.head()

,queries,hf-cross-encoder,rrf,linear-combo-bm25-cosine
0,I'm in the mood for a high-rated action movie ...,"[(Mad Max: Fury Road, -11.244140625), (Toy Sto...","[(The Incredibles, 0.016029143897996357), (Toy...","[(The Incredibles, 0.55239223002), (Toy Story,..."
1,What's a funny animated film about unlikely fr...,"[(Despicable Me, -10.441909790039062), (The In...","[(Monsters, Inc., 0.015524093392945852), (Mada...","[(The Incredibles, 0.45475204289), (Despicable..."
2,Any movies featuring superheroes or extraordin...,"[(The Incredibles, -3.6648073196411133), (The ...","[(The Incredibles, 0.01639344262295082), (The ...","[(The Incredibles, 0.603234915587), (The Aveng..."
3,I want to watch a thrilling movie with spies o...,"[(Inception, -10.843631744384766), (The Incred...","[(Inception, 0.015524093392945852), (Skyfall, ...","[(Skyfall, 0.443840536475), (Despicable Me, 0...."
4,Are there any comedies set in unusual location...,"[(The Incredibles, -11.45376968383789), (Findi...","[(Finding Nemo, 0.015524093392945852), (Madaga...","[(Madagascar, 0.442132219672), (Despicable Me,..."


In [32]:
rankings.loc[12].values

array(['Show me movies set in dystopian or post-apocalyptic worlds',
       list([('Mad Max: Fury Road', -3.4906270503997803), ('Despicable Me', -11.051526069641113), ('The Incredibles', -11.315656661987305), ('Finding Nemo', -10.880638122558594)]),
       list([('The Incredibles', 0.01620835536753041), ('Finding Nemo', 0.013813068651778329), ('Mad Max: Fury Road', 0.011475409836065573), ('Madagascar', 0.01111111111111111)]),
       list([('Mad Max: Fury Road', '0.452238571644'), ('The Incredibles', '0.445061463118'), ('Madagascar', '0.41901564002'), ('Despicable Me', '0.416218408942'), ('Skyfall', '0.411504244804'), ('The Avengers', '0.41121032536'), ('Black Widow', '0.410578364134'), ('The Lego Movie', '0.408463662863'), ('Monsters, Inc.', '0.392220926285'), ('Shrek', '0.390464815497')])],
      dtype=object)

# Wrap up
That's a wrap! Hopefully from this you were able to learn:
- How to implement simple vector search queries in Redis
- How to implement vector search queries with full-text filters
- How to implement hybrid search queries using the Redis aggregation API
- How to perform client-side fusion and reranking techniques